<font size="6">**CircularInclusion-problem for acoustics**</font>
$$
\partial_t p = \text{div} \mathbf{v} \\ 
\partial_t \mathbf{v} = \nabla p 
$$

In [ ]:
!git pull
!git submodule update

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.gridspec as grd
import itertools as it
import re
import pickle
from collections import defaultdict
import itertools
import pandas as pd

if not os.getcwd().endswith("spacetimeproject"):
    os.chdir("..")
    print(os.getcwd())
from mpp.python.mppy.mppy import Mpp

mpp = Mpp(project_name="spacetimeproject")

print("Project:", mpp.dm.PROJECT_ROOT_DIR)
print("MPP:", mpp.dm.MPP_ROOT_DIR)
mpl.rcParams['figure.figsize'] = (15, 10)

In [ ]:
mpp.build()

In [ ]:
hosts = {8:64,9:64}

In [ ]:
def run_and_plot(kwargs, title, host, fig = None, val = ""):
    mpp.reset_data()
    mpp.run(host, "circularInclusion", kwargs)
    mpp.parse_log()
    errs = [["L1_Error", "L2_Error", "LInf_Error"], ["L1_int_Error", "L2_int_Error", "LInf_int_Error"]]
    plt.rcParams['figure.figsize'] = [15, 5]
    shape = (2,len(errs[0]))
    if fig:
        if not fig.axes:
            axs = fig.subplots(*shape)
        else:
            axs = np.array(fig.axes).reshape(shape)
    else:
        fig, axs = plt.subplots(*shape)
    
    fig.suptitle(title, fontsize=15)
    for e1, row in zip(errs,axs):
        for err, ax in zip(e1, row):
            e = mpp.data[err][0]
            ax.set_title(err)
            ax.semilogy(e, marker="x", label=val)

    fig.tight_layout()
    fig.subplots_adjust(top=0.85)
    #fig.legend()
    cols = errs[0]+errs[1] + ["L1_Error_Rate", "L2_Error_Rate", "LInf_Error_Rate"]
    g = [mpp.data.get(name, [[0]])[0] for name in cols]
    dat = list(map(list, itertools.zip_longest(*g, fillvalue=0)))
    if val:
        cols = [c + "_"+ val for c in cols]
    return pd.DataFrame(dat, columns=cols).transpose()

In [ ]:
def run_and_plot_multi(kwargs, title, host):
    ls = [(arg, val) for arg, val in kwargs.items() if type(val) == list]
    print(ls)
    dfs = []
    if ls:
        arg = ls[0][0]
        fig = plt.figure()
        for val in ls[0][1]:            
            kwargs[arg] = val            
            df = run_and_plot(kwargs, title, host, fig=fig, val=val)
            dfs.append(df)
        for ax in fig.axes:
            ax.legend()
    return pd.concat(dfs)

In [ ]:
mpp.mute = True

In [ ]:
run_and_plot_multi({"level":3, 
                      "plevel":3, 
                      "space_deg":1, "time_deg":1, 
                      "refinement_steps":4, 
                      "vtkplot":0, 
                      "set_exact_soltion":0,
                      "theta":-1,
                      "Model": "STDGViscoAcousticAssemble", 
                      "nodalPoints":["EQ", "GL", "GLGL"]},
                   title = "Uniform P-Convergence DGT",    
                   host = hosts
                  )

In [ ]:
run_and_plot_multi(kwargs= {"level":6, 
                      "plevel":3, 
                      "space_deg":1, "time_deg":1, 
                      "refinement_steps":0, 
                      "vtkplot":0, 
                      "set_exact_soltion":0,
                      "theta":-1,
                      "Model": "STDGViscoAcousticAssemble", 
                      "nodalPoints":["EQ", "GL", "GLGL"]},
              title = "Uniform H-Convergence DGT",
              host = hosts)


In [ ]:
run_and_plot_multi(kwargs= {"level":3, 
                      "plevel":3, 
                      "space_deg":1, "time_deg":1, 
                      "refinement_steps":3, 
                      "vtkplot":0, 
                      "set_exact_soltion":0,
                      "theta":0.6,
                      "Model": "STDGViscoAcousticAssemble", 
                      "nodalPoints":["EQ", "GL", "GLGL"]},
              title = "Adaptive P-Convergence DGT",
              host = hosts)

In [ ]:
import tools.seismogram_lib as seislib

In [ ]:
ss = ["s_ST_l_6_probl_3_sDeg_1_tDeg_1_nL_0_pml_0.000000_dt_16_adapt_0_tauz_0.000000_taup_0.100000_kapaf_1.000000_dgInTime",
      "s_ST_l_6_probl_3_sDeg_1_tDeg_1_nL_0_pml_0.000000_dt_32_adapt_0_tauz_0.000000_taup_0.100000_kapaf_1.000000_dgInTime",
      "s_ST_l_6_probl_3_sDeg_1_tDeg_1_nL_0_pml_0.000000_dt_64_adapt_0_tauz_0.000000_taup_0.100000_kapaf_1.000000_dgInTime",
     "s_ST_l_6_probl_3_sDeg_1_tDeg_1_nL_0_pml_0.000000_dt_128_adapt_0_tauz_0.000000_taup_0.100000_kapaf_1.000000_dgInTime"]
seismograms,(d_min,d_max) = seislib.read_seismograms(["build/data/"+k for k in ss], r_step=1)
seislib.show_differences(seismograms, "rel", 0, 1.5)
seislib.plot_seismograms(seismograms, d_min, d_max)

In [ ]:
from vedo import *

vol = load('build/data/vtk/P_0.vtk')
vol2 = load('build/data/vtk/P_1.vtk')

In [ ]:
show(vol, at=0, shape=(1,1))

In [ ]:
show?

In [ ]:
!jupyter nbextension enable --py --sys-prefix k3d

In [ ]:
1+1

In [ ]:
!pdetop

In [ ]:
run_and_plot(kwargs= {"level":3, 
                      "plevel":3, 
                      "space_deg":1, "time_deg":1, 
                      "refinement_steps":4, 
                      "vtkplot":0, 
                      "set_exact_soltion":0,
                      "theta":-1,
                      "Model": "STPGViscoAcousticAssemble", 
                      "nodalPoints":"EQ",
              title = "Uniform P-Convergence Equidistant",
              host = hosts)

In [ ]:
run_and_plot(kwargs= {"level":6, 
                      "plevel":3, 
                      "space_deg":1, "time_deg":1, 
                      "refinement_steps":0, 
                      "vtkplot":0, 
                      "set_exact_soltion":0,
                      "theta":-1,
                      "Model": "STPGViscoAcousticAssemble", 
                      "nodalPoints":"EQ"},
              title = "Uniform H-Convergence Equidistant",
              host = hosts)

In [ ]:
run_and_plot(kwargs= {"level":3, 
                      "plevel":3, 
                      "space_deg":1, "time_deg":1, 
                      "refinement_steps":4, 
                      "vtkplot":0, 
                      "set_exact_soltion":0,
                      "theta":-1,
                      "Model": "STDGViscoAcousticAssemble", 
                      "nodalPoints":"EQ"},
              title = "Uniform P-Convergence DGT Equidistant",
              host = hosts)